In [10]:
from zeep import Client
from zeep.wsse.username import UsernameToken
import pandas as pd
import xml.etree.ElementTree as ET
import re
import html
import os
import csv

# === CONFIGURATION ===
myusername = 'Python1'
mypassword = 'Archer1234!'
myinstance = '330048'
report_id = '16685'

# === Initialize SOAP clients ===
client_general = Client("https://fnz.archerirm.eu/ws/general.asmx?WSDL", wsse=UsernameToken(myusername, mypassword))
client_search = Client("https://fnz.archerirm.eu/ws/search.asmx?WSDL", wsse=UsernameToken(myusername, mypassword))

# === Create session ===
session_token = client_general.service.CreateUserSessionFromInstance(
    userName=myusername,
    password=mypassword,
    instanceName=myinstance
)

# === Field ID to column name mapping ===
column_headers = {
    "4502": "Risk Event ID",
    "4518": "Event Status",
    "22075": "Client",
    "4505": "Risk Event Name",
    "4506": "Risk Event Description",
    "12016": "Event Type",
    "4513": "Where did the event originate",
    "22232": "Monetary Value of the Breach",
    "4604": "Net Financial Loss/Gain",
    "22237": "Is this a gain or loss?",
    "22209": "Primary Root Cause",
    "22210": "Primary Root Cause Sub Category",
    "23823": "Origin legal Entity Selection",
    "22327": "Client Impact",
    "22328": "Client Regulatory Categorisation",
    "22245": "Number of Underlying Customers Affected",
    "22221": "Client Report Sent",
    "22227": "Origin Entity",
    "22939": "Risk Event Category",
    "22238": "FNZ Regulatory Categorisation",
    "4520": "Detection Date and Time",
    "4503": "Created Date",
    "4507": "Event Date",
    "22224": "Date Day 1 Issued",
    "22329": "Date Final Due",
    "22215": "Direct Client /Contractual Impact",
    "22214": "Direct Consumer Harm Impact",
    "22211": "Direct Financial Impact",
    "22212": "Direct Regulatory Impact",
    "22213": "Direct Reputational Impact",
    "4600": "Closed Date",
    "22372": "Origin Entity Selection",
    "22197": "Action Plans",
    "22319": "Helper:Action Plans Closed",
    "22317": "Helper:Total Count of Action Plans",
    "22243": "Process",
    "22961": "Who identified the event?",
    "17241": "Risk Event Owner",
    "22081": "Name of External Fund Manager",
    "4838": "Action Type",
    "22157": "Action Closure Date",
    "4846": "Action Name",
    "4840": "Description",
    "4847": "Action Owner",
    "22161": "Action Owner Team",
    "4849": "Target Completion Date",
    "4839": "Action Status",
    "27819": "Overdue Date",
    "27820": "Days Overdue",
    "4833": "Action ID",
    "22163": "Adjusted Target Completion Date",
    "22196": "Teams Selection",
    "22222": "Calculated Overall Direct Impact",
    "22339": "Is Escalated Root Cause Analysis Required?",
    "22960": "Is there a client impacted by this event?",
    "27912": "Breach Exposure Ceased Date",
    "22338": "Is this a Breach for FNZ?",
    "22362": "Client Regulatory Rule Reference",
    "22241": "Issue Number",
    "4512": "Associated Risks",
    "23818": "Location of Event",
    "17261": "Subject",
    "17262": "Review Status",
    "22863": "Comment Type"
}

# === Clean HTML tags and decode entities ===
def clean_html(raw_html):
    return html.unescape(re.sub(r'<[^>]+>', '', raw_html))

# === Retrieve and parse data ===
all_records = []
field_type_log = {}
current_page = 1

while True:
    report_data = client_search.service.SearchRecordsByReport(
        sessionToken=session_token,
        reportIdOrGuid=report_id,
        pageNumber=current_page
    )
    xml_data = str(report_data)
    root = ET.fromstring(xml_data)

    records = root.findall('.//Record')
    if not records:
        break

    for record in records:
        record_data = {column_name: None for column_name in column_headers.values()}
        for field in record.findall('.//Field'):
            column_id = field.get('id')
            column_name = column_headers.get(column_id)
            if column_name:
                raw_text = None
                field_type = "Unknown"

                list_values = field.findall('.//ListValue')
                if list_values:
                    field_type = "ListValue"
                    raw_text = ', '.join([lv.text for lv in list_values if lv.text])
                else:
                    values_list = field.findall('.//ValuesList/Value')
                    if values_list:
                        field_type = "ValuesList"
                        raw_text = ', '.join([v.text for v in values_list if v.text])
                    else:
                        field_type = "DirectText"
                        raw_text = field.text.strip() if field.text else None

                record_data[column_name] = clean_html(raw_text) if raw_text else None
                field_type_log[column_id] = {
                    "FieldType": field_type,
                    "SampleValue": raw_text
                }

        all_records.append(record_data)

    current_page += 1

# === Convert to DataFrame ===
df_risk_events = pd.DataFrame(all_records)

# Ensure all expected columns are present
for col in column_headers.values():
    if col not in df_risk_events.columns:
        df_risk_events[col] = None

# Reorder columns to match mapping
df_risk_events = df_risk_events[list(column_headers.values())]

# === Save to Excel ===
output_path = os.path.expanduser(r"C:\Users\Victor.Mpofu\OneDrive - FNZ\Documents\archer_risk_events_cleaned.xlsx")
df_risk_events.to_excel(output_path, index=False)
print(f"Cleaned data successfully saved to {output_path}")

# === Save field type diagnostics to CSV ===
diagnostics_path = os.path.expanduser(r"C:\Users\Victor.Mpofu\OneDrive - FNZ\Documents\archer_field_diagnostics.csv")
with open(diagnostics_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Field ID", "Field Type", "Sample Value"])
    for fid, info in field_type_log.items():
        writer.writerow([fid, info["FieldType"], info["SampleValue"]])

print(f"Field diagnostics saved to {diagnostics_path}")

Cleaned data successfully saved to C:\Users\Victor.Mpofu\OneDrive - FNZ\Documents\archer_risk_events_cleaned.xlsx
Field diagnostics saved to C:\Users\Victor.Mpofu\OneDrive - FNZ\Documents\archer_field_diagnostics.csv
